# Timeseries generation with HYP3_ISCE2

(Let ASF do the processing)

In [1]:
import hyp3_sdk as sdk
import asf_search as asf
import geopandas as gpd
import pandas as pd
import os
import copy
from folium import plugins

In [2]:
asf.__version__

'6.7.3'

In [3]:
hyp3 = sdk.HyP3() # 'https://hyp3-api.asf.alaska.edu'
print('User:{user_id} Quota:{quota}'.format(**hyp3.my_info()))

User:scottyhq Quota:{'max_jobs_per_month': 3000, 'remaining': 3000}


## Search for acquisitions

In [4]:
gfa = gpd.GeoDataFrame.from_features( {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          86.870901, 
          27.904352
        ],
        "type": "Point"
      }
    }
  ]
},
 crs=4326   
)

In [6]:
results = asf.search(
                    platform=[asf.PLATFORM.SENTINEL1],
                    processingLevel=asf.BURST,
                    beamMode=asf.BEAMMODE.IW,
                    #relativeOrbit=relorb,
                    polarization='VV',
                    #flightDirection='ASCENDING',
                    intersectsWith=str(gfa.geometry.values[0]),
                    #start='2019-06-01',
                    #end='2023-09-01',
                    #season=[180,270], #first,last day of year June~30*6=180
                    #operaBurstID=['T064_135602_IW1'],
                    )
print(len(results))

gf = gpd.GeoDataFrame.from_features(results.geojson(), crs=4326)

34


In [7]:
gf.iloc[0]

geometry           POLYGON ((86.15505 27.926617, 86.197816 27.731...
beamModeType                                                      IW
browse                                                          None
bytes                                                      131033760
centerLat                                                  27.902804
centerLon                                                  86.611314
faradayRotation                                                 None
fileID                   S1_023790_IW1_20240111T121428_VV_5827-BURST
flightDirection                                            ASCENDING
groupID                                S1A_IWDV_0084_0090_052059_012
granuleType                                                     None
insarStackId                                                    None
md5sum                                                          None
offNadirAngle                                                   None
orbit                             

In [8]:
gf['burstId'] = gf.burst.str['fullBurstID']
gf.groupby('burstId').sceneName.count()

burstId
012_023790_IW1    18
121_258661_IW2    16
Name: sceneName, dtype: int64

In [9]:
# Pick a single burst ID to work with
burst = '012_023790_IW1'

In [10]:
# Look at timeline of acquisition
gfb = gf[gf.burstId == burst]
gfb.loc[:,'datetime'] = pd.to_datetime(gfb.startTime)

/Users/scott/miniforge3/envs/hyp3/lib/python3.12/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [11]:
m = gf.iloc[[0]].explore()
minimap = plugins.MiniMap(width=200, height=100)
m.add_child(minimap)